# Data cleaning

In [1]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats


In [2]:
# set options
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [3]:
# load data and check the shape
loan_data = pd.read_csv("Loan_data.csv", low_memory = False)
loan_data.shape

(42536, 151)

In [5]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,NaN,Aug-2018,739.0,735.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [6]:
# Drop columns with all missing values
loan_data.dropna(axis = 1, how = 'all', inplace = True)
loan_data.shape

(42536, 69)

In [7]:
# In columns loan_status, there are 9 unique values. We will keep only 'Charged Off' and 'Fully Paid' values.
loan_data = loan_data[(loan_data.loan_status=='Charged Off')|
          (loan_data['loan_status']=='Fully Paid')]
loan_data.shape

(39786, 69)

In [8]:
# Drop columns with more than 60% missing values
threshold = 0.6
loan_data = loan_data[loan_data.columns[loan_data.isnull().mean() < threshold]]
loan_data.shape

(39786, 60)

In [9]:
# Drop columns with only one unique value
loan_data = loan_data[loan_data.columns[loan_data.nunique() > 1]]
loan_data.shape

(39786, 47)

In [10]:
# Drop columns with unimportant information
loan_data.drop(['id', # unique value
                'funded_amnt', # same as loan_amnt
                'funded_amnt_inv', # same as loan_amnt
                'emp_title', # too many unique values
                'total_rec_late_fee', # almost all values are 0
                'recoveries', # almost all values are 0
                'addr_state', # state not adding any value
                'zip_code', # zip code not adding any value
                'url', # unique value
                'desc', # too many unique values
                'debt_settlement_flag', # almost all values are N
                'title', # too many unique values
                'pub_rec', #pub_rec_buncrupcy is more important
                'collection_recovery_fee' # almost all values are 0
                ], axis=1, inplace = True) 

In [12]:
# Drop colums fico_range_high and keep fico_range_low as fico_range_low and fico_range_high are same
loan_data.drop(['fico_range_high'], axis=1, inplace = True)
loan_data.rename(columns = {'fico_range_low':'fico_range'}, inplace = True)

In [11]:
# Drop columns with information from future
loan_data.drop(['last_pymnt_d', 
                'last_pymnt_amnt', 
                'last_credit_pull_d', 
                'last_fico_range_high', 
                'last_fico_range_low',
                'total_pymnt', 
                'total_pymnt_inv',
                'total_rec_prncp',
                'total_rec_int',
                'installment',
                'revol_util'], axis=1, inplace = True)

In [13]:
loan_data.head()

,loan_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range,inq_last_6mths,open_acc,revol_bal,total_acc,pub_rec_bankruptcies
0,5000.0,36 months,10.65%,B,B2,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,27.65,0.0,Jan-1985,735.0,1.0,3.0,13648.0,9.0,0.0
1,2500.0,60 months,15.27%,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,1.00,0.0,Apr-1999,740.0,5.0,3.0,1687.0,4.0,0.0
2,2400.0,36 months,15.96%,C,C5,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,8.72,0.0,Nov-2001,735.0,2.0,2.0,2956.0,10.0,0.0
3,10000.0,36 months,13.49%,C,C1,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,20.00,0.0,Feb-1996,690.0,1.0,10.0,5598.0,37.0,0.0
4,3000.0,60 months,12.69%,B,B5,1 year,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,other,17.94,0.0,Jan-1996,695.0,0.0,15.0,27783.0,38.0,0.0


In [14]:
# Chenge the data type of columns
loan_data['term'].replace({' 36 months':36, 
                           ' 60 months':60}, inplace = True)
# Rename the column
loan_data.rename(columns = {'term':'credit_month'}, inplace = True)


In [15]:
loan_data['pub_rec_bankruptcies'].value_counts()

pub_rec_bankruptcies
0.0    37404
1.0     1678
2.0        7
Name: count, dtype: int64

In [16]:
# Replace 2 with 1 as its more frequent
loan_data['pub_rec_bankruptcies'].replace({2:1}, inplace = True)

In [17]:
loan_data['home_ownership'].value_counts()

home_ownership
RENT        18918
MORTGAGE    17703
OWN          3064
OTHER          98
NONE            3
Name: count, dtype: int64

In [18]:
# Replace NONE with OTHER 
loan_data['home_ownership'].replace({'NONE':'OTHER'}, inplace = True)

In [19]:
# Replace percentage with float
loan_data['int_rate'] = loan_data['int_rate'].str.rstrip('%').astype('float') / 100.0

In [20]:
# Category encoding
loan_data['emp_length'].replace({'10+ years':10, 
                            '1 year':1, 
                            '3 years':3, 
                            '8 years':8, 
                            '9 years':9, 
                            '4 years':4, 
                            '5 years':5,
                            '6 years':6,
                            '2 years':2,
                            '7 years':7,
                            '< 1 year':0}, inplace = True)

In [21]:
# Replace text date with datetime
loan_data['issue_d'] = pd.to_datetime(loan_data['issue_d'], format='%b-%Y')
loan_data['earliest_cr_line'] = pd.to_datetime(loan_data['earliest_cr_line'], format='%b-%Y')

In [22]:
# Less important categorical values are grouped together as 'other'
def categorize(name):
    if name == 'debt_consolidation':
        return 'debt_consolidation'
    elif name == 'home_improvement':
        return 'home_improvement'
    elif name == 'credit_card':
        return 'credit_card'
    elif name == 'major_purchase':
        return 'major_purchase'
    else:
        return 'other'

loan_data['purpose'] = loan_data['purpose'].apply(categorize)

In [23]:
loan_data.head()

,loan_amnt,credit_month,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range,inq_last_6mths,open_acc,revol_bal,total_acc,pub_rec_bankruptcies
0,5000.0,36,0.1065,B,B2,10.0,RENT,24000.0,Verified,2011-12-01,Fully Paid,credit_card,27.65,0.0,1985-01-01,735.0,1.0,3.0,13648.0,9.0,0.0
1,2500.0,60,0.1527,C,C4,0.0,RENT,30000.0,Source Verified,2011-12-01,Charged Off,other,1.00,0.0,1999-04-01,740.0,5.0,3.0,1687.0,4.0,0.0
2,2400.0,36,0.1596,C,C5,10.0,RENT,12252.0,Not Verified,2011-12-01,Fully Paid,other,8.72,0.0,2001-11-01,735.0,2.0,2.0,2956.0,10.0,0.0
3,10000.0,36,0.1349,C,C1,10.0,RENT,49200.0,Source Verified,2011-12-01,Fully Paid,other,20.00,0.0,1996-02-01,690.0,1.0,10.0,5598.0,37.0,0.0
4,3000.0,60,0.1269,B,B5,1.0,RENT,80000.0,Source Verified,2011-12-01,Fully Paid,other,17.94,0.0,1996-01-01,695.0,0.0,15.0,27783.0,38.0,0.0


In [25]:
# Save the cleaned data and use it for further analysis
loan_data.to_csv("c_data.csv")